# Proyecto Cartera

- Leer posiciones de activos de hoja google "Activos".
- A partir de las posiciones, Leer valoraciones de la web -> grabar hoja google "Valoraciones".

### Leer posiciones de activos de hojas google.

Hoja google -> **Cuentas y Cartera 2022**       Contiene toda la inf., Cuentas y Cartera

Trabajando con hoja copia -> **"Cartera limpia"**
hoja:  https://docs.google.com/spreadsheets/d/1qGM6LUm9FO_KyVwv1-h9j9bS3c6uqfJi9-iH4qBJh1w/edit#gid=1100996300

**Para poder ver la hoja**  tiene que estar compartida con:

appservicio@python-tutorial3.iam.gserviceaccount.com

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#  Ignora los warnings que muestra los modelos
import warnings
warnings.filterwarnings('ignore')

#   Definir el límite de acceso, el scope = alcance, acceso a hojas de cálculo y drive
alcance =  ['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive']

Credenciales = ServiceAccountCredentials.from_json_keyfile_name('Credenciales.json', alcance)

cliente = gspread.authorize(Credenciales)

#  Lectura de hoja google "Cartera limpia" => pestaña "Activos"
hoja = cliente.open("Cartera limpia")
activos = hoja.worksheet('Activos')
lista_activos = activos.get_all_values()    # vuelca en Lista

#  Defino función para transformar los números de la hoja con puntos y coma y símbolo euro
def ExtraerNum(texto):
    if texto == '':
        valor = 0
    else:
        caracterEliminar = '. €+EURSD'
        for x in range(0, len(caracterEliminar)):
            texto = texto.replace(caracterEliminar[x], '')
    
        valor = float(texto.replace(",", "."))
    return valor

#  Proceso la lista de activos analizando los datos y transformando campos numéricos
#      Grabamos el resultado en un DataFrame
import pandas as pd

cont = 0
datos = list()
for linea in lista_activos:
    try:
        cont += 1
        if cont == 1:
            indice = linea[0:14]
            print(indice)
        if cont > 1 and linea[11] != '':
            #print("P. Compra: ", linea[10], " -> ", linea[0:3] + linea[6:12])
            compra = ExtraerNum(linea[7])
            # fecha = linea[9]
            num = ExtraerNum(linea[10])
            traspaso = ExtraerNum(linea[11])
            valor = ExtraerNum(linea[12])
            datos.append(linea[0:7] + [compra] + linea[8:10] + [num, traspaso, valor] + [linea[13]])
    except ValueError:
        print("Error línea: ", cont, " línea[7:12]-> ", linea[7:14])


#  Grabo la información extraída: índice y datos en DataFrame
df = pd.DataFrame(datos, columns=indice)

['Activo', 'ISIN', 'MorningStar', 'Producto', 'Bloque', 'Tipo', 'Categoría', 'P. Compra', 'Entidad', 'Fecha', 'Num.', 'P. Traspaso', 'Valoración', 'Abierta']


### Información del DataFrame leído de Activos -> df -> HISTORIAL (foto fija en vertical)

- Agrupar la información por Activo (groupby) -> Bloque, Tipo y Activo  => Calculamos sumatorios 

In [2]:
dfActivos = df[df['Abierta'] == 'A']
dfActivos = dfActivos.groupby(['Bloque', 'Tipo','Activo']).sum()
dfActivos.reset_index(inplace=True)

#  Grabamos en pestaña "Historial"
historial = hoja.worksheet('Historial')
cabecera = dfActivos.columns.values.tolist()
datos = dfActivos.values.tolist()
historial.update([cabecera] + datos)

{'spreadsheetId': '1qGM6LUm9FO_KyVwv1-h9j9bS3c6uqfJi9-iH4qBJh1w',
 'updatedRange': 'Historial!A1:N21',
 'updatedRows': 21,
 'updatedColumns': 14,
 'updatedCells': 294}

### Información del DataFrame leído de Activos -> df -> CARTERA (foto fija en horizontal)

- Partimos del dfActivos (anterior) para formaterar como hoja excel Cartera.

In [3]:
cartera = dfActivos
cartera = cartera.drop(columns=['P. Compra','Num.','P. Traspaso'], axis=1)
cartera = cartera.transpose()
cartera.reset_index(inplace=True)

import datetime as dt

#  Arreglamos columna Index
cartera.loc[0, 'index'] = ''
cartera.loc[1, 'index'] = ''
cartera.loc[3, 'index'] = str(dt.date.today())

In [4]:
#  Recorrer las filas de Bloques y Tipos y crear columnas para totales
#  Acumulamos el valor en la última línea
bloque = cartera.iloc[0][0]
tipo = cartera.iloc[1][0]
suma = 0
ins = 0
n = 0
total = 0
totalCartera = 0
tamaño = len(cartera.iloc[1])
print(tamaño)
for i in range(tamaño - 1):
    print("> ", i, str(cartera.iloc[0][i]), str(cartera.iloc[1][i]), str(cartera.iloc[2][i]))
    if tipo == str(cartera.iloc[1][i]) and bloque == str(cartera.iloc[0][i]):
        n += 1
        suma += cartera.iloc[3][i]
    else:
        if n > 1:
            total += 1
            nombre = 'Total' + str(total)
            ins += 1
            print("  > ", nombre)
            columna = [bloque, tipo, 'VALOR', round(cartera.iloc[3][i-n+ins : i+ins].sum())]
            cartera.insert(loc=i+ins, column=nombre, value=columna)
            print(round(cartera.iloc[3][i-n+ins : i+ins].sum()))
        
        if bloque != str(cartera.iloc[0][i]):
            totalCartera += 1
            nombre = 'Cartera' + str(totalCartera)
            ins += 1
            columna = [bloque, 'TOTAL', 'TOTAL', suma]
            cartera.insert(loc=i + ins, column=nombre, value=columna)
            suma = cartera.iloc[3][i]
        else:
            suma += cartera.iloc[3][i]
        
        tipo =  str(cartera.iloc[1][i])
        bloque = str(cartera.iloc[0][i])
        n = 1

i = i+1
total += 1
nombre = 'Total' + str(total)
ins += 1
columna = [bloque, tipo, 'VALOR', round(cartera.iloc[3][i-n+ins : i+ins].sum())]
cartera.insert(loc=i+ins, column=nombre, value=columna)

totalCartera += 1
nombre = 'Cartera' + str(totalCartera)
ins += 1
columna = [bloque, 'TOTAL', 'TOTAL', suma]
cartera.insert(loc=i + ins, column=nombre, value=columna)

21
>  0 CARTERAS CASH MyInvestor Cuenta


TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [5]:
#  Grabamos en pestaña "Cartera"
hCartera = hoja.worksheet('Cartera')
#cabecera = cartera.columns.values.tolist()
datos = cartera.values.tolist()
hCartera.update(datos)

WorksheetNotFound: Cartera

### Generar hoja Valoración

- A partir de los activos, nos quedamos con 2 columnas: Activo y ISIN
- Le añadimos columna Valor a ceros
- Crear función que recupere el valor del activo por el ISIN

-> Grabamos los datos en hoja Valoración

In [6]:
import requests
import bs4
from bs4 import BeautifulSoup
import re


#  Eliminamos activos duplicados
df_valor = df[['Activo', 'Producto', 'ISIN', 'MorningStar']].drop_duplicates()


def LeerPrecio(tipo, activo, ticket):
    if tipo == 'FONDO' or tipo == 'ETF':
        if tipo == 'FONDO':
            urlFondo = "https://www.morningstar.es/es/funds/snapshot/snapshot.aspx?id="
            url = urlFondo + ticket
        else:
            url1 = "https://www.morningstar.es/es/etf/snapshot/snapshot.aspx?id="
            url2 = "&InvestmentType=FE"
            ticket = "0P0000YXKB"       # Vanguard S&P 500 UCITS ETF (EUR) | VUSA
            url = url1 + ticket + url2

        html = requests.get(url)
        soup = BeautifulSoup(html.text, "lxml")

        try:
            precio = str(soup.find(class_="line text"))
            precio = precio[precio.find('>') + 1:]
            valor = ExtraerNum(precio[:precio.find('<')])
        except:
            print(url)
            print(" > Línea precio: ", precio)
            print(" -> Error extraer precio MorningStar: ", ticket, " Activo: ", activo)
            valor = 0    
    
    elif tipo == 'ACCIÓN':
        url1 = "https://tools.morningstar.es/es/stockreport/default.aspx?Site=es&id="
        url2 = "&LanguageId=es-ES&SecurityToken="
        url3 = "]3]0]E0WWE$$ALL"
        url = url1 + ticket + url2 + ticket + url3

        html = requests.get(url)
        soup = BeautifulSoup(html.text, "lxml")

        try:
            for l in soup.find(class_="price"):
                valor = ExtraerNum(l)
        except:
            print(" -> Error extraer precio MorningStar: ", ticket, " Activo: ", activo)
            valor = 0

    elif tipo == 'P.PENSIÓN':
        url1 = "https://tools.morningstar.es/2nhcdckzon/snapshot/snapshot.aspx?SecurityToken="
        url2 = "]2]0]FOESP$$PEN"
        url = url1 + ticket + url2
        
        html = requests.get(url)
        soup = BeautifulSoup(html.text, "lxml")

        try:
            cont = 0
            for l in soup.find_all(class_="value_div text-right text"):
                cont += 1
                if cont == 4:
                    precio = [float(s) for s in re.findall(r'-?\d+\.?\d*', str(l))]
                    valor = precio[0]
                    break
        except:
            print(" -> Error extraer precio MorningStar: ", ticket, " Activo: ", activo)
            valor = 0

    else:
        valor = 1
    return valor

def Valorar(entrada):
    activo = entrada[0]
    producto = entrada[1]
    isin = entrada[2]
    morningstar = entrada[3]
    if morningstar == '':
        valor = 0
    else:
        valor = LeerPrecio(producto, activo, morningstar)
    return valor

#  Usamos la función Valorar con todos los parámetros del Dataframe
df_valor['Valor'] = df_valor.apply(Valorar, axis=1)
df_valor.head()

#  Grabamos los datos de Valoración en su pestaña "Valoración"
valoracion = hoja.worksheet('Valoración')
cabecera = df_valor.columns.values.tolist()
datos = df_valor.values.tolist()
valoracion.update([cabecera] + datos)

{'spreadsheetId': '1qGM6LUm9FO_KyVwv1-h9j9bS3c6uqfJi9-iH4qBJh1w',
 'updatedRange': "'Valoración'!A1:E34",
 'updatedRows': 34,
 'updatedColumns': 5,
 'updatedCells': 170}

In [7]:
df_valor

,Activo,Producto,ISIN,MorningStar,Valor
0,BBVA,ACCIÓN,BME:BBVA,0P0000A5RZ,6.54
5,Indexa Fondo,CARTERA,,CARTERA,1.00
7,Cartera Indie,CARTERA,,CARTERA,1.00
8,Vanguard U.S. 500,FONDO,IE0032126645,F0GBR04UOL,47.97
24,Azvalor Internacional,FONDO,ES0112611001,F00000WI0D,214.00
28,R4 Global Allocation,FONDO,ES0116848005,F00000LICF,36.37
31,Global Technology A,FONDO,LU0099574567,F0GBR04D20,52.21
40,Vanguard ESG EUR,FONDO,IE00B5456744,F00000LY5Q,319.20
44,Vanguard G. Small-Cap,FONDO,IE00B42W3S00,F00000T1HQ,295.79
48,iShares Emerging EUR,FONDO,IE00BYWYCC39,F00000YZAJ,11.60


### Pruebas obtención de valoración

##### **Yahoo Finance** -> DataReader(ticket, "yahoo", fechadesde, fechahasta)

- Los precios de fondos van con un día de retraso con respecto a **Investing** o **MorningStar**
- No encuentra muchos activos, por ticket.

In [41]:
import datetime as dt
from pandas_datareader import data

def Valorar(entrada):
    activo = entrada[0]
    isin = entrada[1]
    if isin == '':
        valor = 0
    else:
        fecha = dt.date.today()
        try:
            df2 = data.DataReader(isin, "yahoo", fecha, fecha)
            valor = df2.Close[0]
        except:
            print(" -> No encontrado en yahoo: ", isin, " Activo: ", activo)
            valor = 0
    return valor

##### **yfinance** - Pruebas

In [4]:
import yfinance as yf

activo = 'BBVA.MC'
activo = '0P00000SUJ.F'

valor = yf.Ticker(activo)
hist = valor.history(period="10d")
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-12-05 00:00:00+01:00,44.870602,44.870602,44.870602,44.870602,0,0,0
2022-12-06 00:00:00+01:00,44.331001,44.331001,44.331001,44.331001,0,0,0
2022-12-07 00:00:00+01:00,44.244301,44.244301,44.244301,44.244301,0,0,0
2022-12-08 00:00:00+01:00,44.404301,44.404301,44.404301,44.404301,0,0,0
2022-12-09 00:00:00+01:00,44.099899,44.099899,44.099899,44.099899,0,0,0


##### **Investing** -> Usar librería

- No funciona -> ERR#0015: error 403, try again later.

In [ ]:
import investpy as inv

fondo = 'Vanguard U.s. 500 Stock Index Fund Investor Eur Accumulation'
pais = 'Ireland'
#  1) Uso función get_fund_historical_data, parámetros: fund, country, from_date y to_date
#datos = inv.get_fund_historical_data(fondo, pais, '17/10/2022', '18/10/2022')
#  Devuelve error ->  ERR#0015: error 403, try again later.

#  2) Uso función get_fund_information, parámetros: fund, country
#datos = inv.get_fund_information(fondo, pais)
#  Devuelve error ->  ERR#0015: error 403, try again later.

#  3) Uso función get_fund_recent_data, parámetros: fund, country, interval 
datos = inv.get_fund_recent_data(fondo, pais, interval='Monthly')
print(datos)

##### **Investing.com** -> web scraping

- No se puede hacer web scraping (bloquea)

##### **MorningStar** -> web scraping

Prueba Vanguard US500 stock Index 

- Lectura de página correcta
- Leer precio correcto

In [6]:
import requests
import bs4
from bs4 import BeautifulSoup

In [65]:
urlFondo = "https://www.morningstar.es/es/funds/snapshot/snapshot.aspx?id="
isin = "F0GBR04UOL"
isin = "F00000WI0D"
html = requests.get(urlFondo + isin)
soup = BeautifulSoup(html.text, "lxml")

precio = str(soup.find(class_="line text"))     # devuelve <td class="line text">EUR 44,53</td>
precio = precio[precio.find('>') + 1:]
precio = ExtraerNum(precio[:precio.find('<')])
print("Precio númerico: ", precio) 

Precio númerico:  201.54


Prueba Acción BBVA

- Lectura de página correcta
- Leer precio correcto

In [66]:
url1 = "https://tools.morningstar.es/es/stockreport/default.aspx?Site=es&id="
url2 = "&LanguageId=es-ES&SecurityToken="
url3 = "]3]0]E0WWE$$ALL"
bbva = "0P0000A5RZ"
sant = "0P0000A5JA"
url = url1 + bbva + url2 + bbva + url3

html = requests.get(url)
soup = BeautifulSoup(html.text, "lxml")

lineas = soup.find(class_="price")
for l in lineas:
    valor = ExtraerNum(l)
print("Precio númerico: ", valor) 

Precio númerico:  4.97


Prueba Planes de Pensiones -> Indexa Mas Rentabilidad Acciones PP

In [43]:
url = "https://tools.morningstar.es/2nhcdckzon/snapshot/snapshot.aspx?SecurityToken=F00000Y5QI]2]0]FOESP$$PEN"
indexaBonos = "F00000Y5QH"
indexaAcciones = "F00000Y5QI"

url1 = "https://tools.morningstar.es/2nhcdckzon/snapshot/snapshot.aspx?SecurityToken="
url2 = "]2]0]FOESP$$PEN"
url = url1 + indexaAcciones + url2

html = requests.get(url)
soup = BeautifulSoup(html.text, "lxml")

import re

lineas = soup.find_all(class_="value_div text-right text")
cont = 0
for linea in lineas:
    cont += 1
    print(cont)
    if cont == 4:
        precio = [float(s) for s in re.findall(r'-?\d+\.?\d*', str(linea))]
        valor = precio[0]
        break
print("Precio númerico: ", valor) 

1
2
3
4
Precio númerico:  15.17


In [10]:
url = "https://www.morningstar.es/es/etf/snapshot/snapshot.aspx?id=0P0000YXKB&InvestmentType=FE"
url1 = "https://www.morningstar.es/es/etf/snapshot/snapshot.aspx?id="
url2 = "&InvestmentType=FE"
ticket = "0P0000YXKB"

url = url1 + ticket + url2

html = requests.get(url)
soup = BeautifulSoup(html.text, "lxml")

import re

precio = str(soup.find(class_="line text"))     # devuelve <td class="line text">EUR 44,53</td>
precio = precio[precio.find('>') + 1:]
valor = ExtraerNum(precio[:precio.find('<')])
print("Precio númerico: ", valor) 

Precio númerico:  71.77


### Graba la web en fichero plano

In [9]:
#  Grabar la web en fichero plano
with open('.\\Ficheros\\ETF us500.txt', "w", encoding = 'utf-8') as file:
    file.write(str(soup.prettify()))